# Финансовый ассистент на базе GigaChat и LangGraph

In [1]:
# Установка необходимых библиотек
!pip install -q langchain langchain_gigachat langchain_community langgraph

# Импорт необходимых модулей и классов
from langchain_gigachat.chat_models import GigaChat
from langchain.agents import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Literal
import uuid
import time

# Получение токена доступа из переменных окружения Google Colab
from google.colab import userdata
gigatoken = userdata.get('GIGA_TOKEN')

# Инициализация модели GigaChat с заданными параметрами
giga = GigaChat(
    credentials=gigatoken,
    scope="GIGACHAT_API_CORP",
    model="GigaChat-2-MAX",
    verify_ssl_certs=False,
    profanity_check=False,
    temperature=0.7,
    top_p=0.36,
    repetition_penalty=1,
    max_tokens=1024
)

# "База данных" банковских карт
cards_db = {
    "2202208XXXX11824": {"type": "МИР", "blocked": False},
    "4508103XXXX14732": {"type": "VISA", "blocked": False},
}

# "База данных" счетов с привязанными картами
accounts_db = {
    "user_123": {
        "balance": 5000.0,
        "transactions": [],
        "cards": ["2202208XXXX11824", "4508103XXXX14732"]
    }
}

@tool
def get_cards(account_id: str = "user_123") -> dict:
    """
    Возвращает состояние банковских карт, привязанных к счету пользователя.
    """
    account = accounts_db.get(account_id)
    if not account:
        return {"error": "Счет не найден. Пожалуйста, проверьте правильность идентификатора."}
    account_cards = account.get("cards", [])
    linked_cards = {card_id: cards_db.get(card_id, "Информация о карте не найдена") for card_id in account_cards}
    print(">>> вызов get_cards")
    return linked_cards

@tool
def block_card(card_id: str, reason: Literal["lost", "not_used", "stolen"], account_id: str = "user_123") -> str:
    """
    Блокирует карту пользователя по номеру карты (card_id), если она привязана к счету.
    reason - причина блокировки: 'lost' (потеряна), 'not_used' (не используется) или 'stolen' (украдена).
    """
    account = accounts_db.get(account_id)
    if not account:
        return "Счет не найден. Пожалуйста, проверьте правильность идентификатора."
    if card_id not in account.get("cards", []):
        return f"Карта {card_id} не привязана к вашему счету."
    print(f">>> вызов block_card({card_id}, {reason})")
    if card_id in cards_db:
        cards_db[card_id]["blocked"] = True
        return f"Карта {card_id} успешно заблокирована."
    else:
        return f"Неизвестная карта {card_id}."

@tool
def get_balance(account_id: str = "user_123") -> str:
    """
    Возвращает текущий баланс счета пользователя.
    """
    account = accounts_db.get(account_id)
    if account:
        print(">>> вызов get_balance")
        return f"Текущий баланс вашего счета: {account['balance']} руб."
    else:
        return "Счет не найден. Пожалуйста, проверьте правильность идентификатора."

@tool
def transfer_money(recipient: str, amount: float, account_id: str = "user_123") -> str:
    """
    Переводит указанную сумму на счет получателя и обновляет историю транзакций.
    """
    account = accounts_db.get(account_id)
    if not account:
        return "Счет отправителя не найден. Пожалуйста, проверьте правильность идентификатора."
    if account["balance"] < amount:
        return "Недостаточно средств для выполнения перевода."

    account["balance"] -= amount
    account["transactions"].append(f"Перевод {amount} руб. на счет {recipient}")
    print(f">>> вызов transfer_money({account_id}, {recipient}, {amount})")
    return f"Перевод {amount} руб. на счет {recipient} выполнен успешно."

@tool
def transaction_history(account_id: str = "user_123") -> dict:
    """
    Возвращает историю транзакций по счету пользователя.
    """
    account = accounts_db.get(account_id)
    if account:
        print(">>> вызов transaction_history")
        return {"transactions": account["transactions"]}
    else:
        return {"error": "Счет не найден. Пожалуйста, проверьте правильность идентификатора."}

# Системное сообщение, определяющее поведение ассистента
system = """Ты – финансовый ассистент, помогающий пользователю управлять банковским счетом и привязанными к нему картами.
Пожалуйста, уточняй детали перед выполнением перевода, блокировкой карт, проверкой баланса или выдачей истории транзакций.
Если пользователь просит заблокировать карту, обязательно уточни у него причину блокировки. Не блокируй карту, пока не узнаешь причину блокировки.
"""

# Создание агента с использованием модели GigaChat и определённых инструментов
agent = create_react_agent(
    giga,
    tools=[get_cards, block_card, get_balance, transfer_money, transaction_history],
    checkpointer=MemorySaver(),
    prompt=system
)

# Конфигурация агента с уникальным идентификатором сессии
config = {"configurable": {"thread_id": str(uuid.uuid4())}}


In [2]:
# Интерактивный цикл для общения с агентом
while True:
    query = input("User: ")
    if query == "" or query.lower() == "q":
        break
    resp = agent.invoke({"messages": [{"role": "human", "content": query}]}, config)
    print(f'Bot: {resp["messages"][-1].content}')
    time.sleep(0.5)

User: cколько денег у меня? 
>>> вызов get_balance
Bot: На Вашем счету 5000 рублей.
User: а какие карты? 
>>> вызов get_cards
Bot: У Вас есть две карты: карта МИР с последними цифрами 11824 и карта VISA с последними цифрами 14732. Обе карты активны.
User: переведи 2000 рублей Петрову Ивану и 3000 рублей Сидорову Геннадию
>>> вызов transfer_money(user_123, Петров Иван, 2000.0)
>>> вызов transfer_money(user_123, Сидоров Геннадий, 3000.0)
Bot: Переводы выполнены успешно.
User: Заблокируй все карты, так как они мне больше не нужны
>>> вызов get_cards
>>> вызов block_card(2202208XXXX11824, not_used)
>>> вызов block_card(4508103XXXX14732, not_used)
Bot: Все карты успешно заблокированы.
User: 
